# 第 4 章 モデルの作成

## 4.1 モデルとは何か

## 4.1.1 モデルとは何か?
- モデルの定義: 入力データを特徴量とし, 出力を予測値とする変換器.
    - 例: ランダムフォレスト, ニューラルネット.
- 復習
    - 目的変数: 予測したいラベルや値. 例: 1 ヶ月以内の有料機能利用状況.
    - 特徴量: 目的変数の予測に使える量. 例: 有料機能を使う様々なユーザー属性.
- 分析コンペでは「教師あり学習」が基本: 教師あり学習に関して, 必要なら以下の補足を参考にする.
- ハイパーパラメータ: モデルが持つ, 学習前に指定するパラメータ.
    - 学習の方法, 速度, どれだけ複雑なモデルにするかが決まる.
    - モデルの精度に影響する.
    - ハイパーパラメータの例: 正則化 (後述) の強さを調整する.
    - チューニングについては 6 章で議論する.
- 自分用のメモ: パラメータとハイパーパラメータの違い
    - パラメータ: 正規分布の平均や分散. これ自体が推測の対象
    - ハイパーパラメータ: 上で書いたように正則化の強さなどの調整弁.

## 補足: 教師あり学習
- 教師あり学習: 目的変数があるデータからモデルを学習させ, 目的変数がないデータに対して予測する.
- 半教師あり学習
    - 目的変数があるデータだけではなく目的変数がないデータについてもモデルの学習に活用する.
    - 少量のラベルありデータを使って大量のラベルなしデータを学習に活かす.
    - 人間の学習と似ている
        - 様々なもののラベル (例えば「これはネコ」「これはイヌ」) を他人から与えられる.
        - 後は自分でたくさんのネコやイヌを見てどんどん学習して認識できるようになっていく.
- 教師なし学習
    - 目的変数がないデータからデータ内のパターンを推測する.
    - データから共通する特徴を持つグループを見つけたり, データを特徴づける情報を抽出したりする.
    - 例
        - 類似データのグルーピング (クラスタリング)
        - 本質的なデータを抽出する (次元削減)
            - 参考: https://aizine.ai/glossary-unsupervised-learning/
            - A さんに対するイメージのアンケートから「美人」を導出する.

## 4.1.2 モデル作成の流れ

## モデルの学習と予測
- (ハイパーパラメータは適切な値がわかっているとする)
- モデルの種類とハイパーパラメータを指定する
- 学習データと目的変数を与えて学習させる
- テストデータを与えて予測させる
- 参考コードは以下の通り

In [ ]:
import numpy as np
import pandas as pd

train = pd.read_csv('~/codes/kagglebook/input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('~/codes/kagglebook/input/sample-data/test_preprocessed.csv')

import xgboost as xgb

class Model:
    def __init__(self, params=None):
        self.model = None
        if params is None:
            self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y):
        params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
        params.update(self.params)
        num_round = 10
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        self.model = xgb.train(params, dtrain, num_round)

    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred

params = {'param1': 10, 'param2': 100}
model = Model(params)
model.fit(train_x, train_y)
pred = model.predict(test_x)

## 出てくる行列の次数
- 学習データのレコード数を $n_{tr}$, テストデータのレコード数を $n_{te}$, 特徴量の列数を $n_f$ とする.
- 学習データは $(n_{tr}, n_{f})$ 次の行列
- 目的変数は $n_{tr}$ 個のベクトル (配列)
- テストデータは $(n_{te}, n_f)$ 次の行列
- 予測値は $n_{te}$ 個のベクトル (配列)

## モデルの評価 (バリデーション)
- モデルの良し悪しを評価する: 詳しくは 第 5 章.
- 学習に使ったデータに関してモデルは「正解を知っている」.
- 一方, 知りたいのはあくまで未知のデータに対する予測能力.
- モデル評価用に一部のデータをバリデーション (検証用) データとして分けて評価に使う.
- モデルによっては学習時に学習データとともにバリデーションデータを与えられる.
    - 学習進度をモニタリングできる.

In [ ]:
import numpy as np
import pandas as pd

train = pd.read_csv('~/codes/kagglebook/input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('~/codes/kagglebook/input/sample-data/test_preprocessed.csv')

import xgboost as xgb
class Model:
    def __init__(self, params=None):
        self.model = None
        if params is None:
            self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y):
        params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
        params.update(self.params)
        num_round = 10
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        self.model = xgb.train(params, dtrain, num_round)

    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred

from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]

tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

model = Model(params)
model.fit(tr_x, tr_y)
va_pred = model.predict(va_x)
score = log_loss(va_y, va_pred)
print(f'logloss: {score:.4f}')

## hold-out 法
- 一部のデータをバリデーションデータとして取り分ける.
- 欠点: モデルの学習・評価に使えるデータが少なくなる.
- 克服のために使われるのがクロスバリデーション.

## クロスバリデーション
- 学習データを分割する: 分割したそれぞれの固まりを fold と呼ぶ.
- そのうちの 1 つをバリデーションデータ, 残りを学習データとして学習・評価し,
  バリデーションデータでのスコアを求める.
- 分割した数だけバリデーションデータを変えて学習・評価する.
- それらのスコアの平均でモデルを評価する.

In [ ]:
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

scores = []
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    model = Model(params)
    model.fit(tr_x, tr_y)
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)
    scores.append(score)

print(f'logloss: {np.mean(scores):.4f}')

## P.222 分析コンペでの学習・評価・予測の流れ
- モデルを評価するバリデーションの枠組みを作り, 評価のフィードバックを改善に活かす.
- モデルの作成・評価の 1 サイクルは次の通り
    - モデルの種類とハイパーパラメータを指定してモデルを作る
    - 学習データを与えてモデルを学習させ, バリデーションしてモデルを評価する
    - 学習したモデルでテストデータに対して予測する
- モデル改善
    - 特徴量を追加・変更する
    - ハイパーパラメータを変える
    - モデルを変える
- バリデーションはふつうクロスバリデーション.
    - スタッキングのようなアンサンブル (7 章参考: 複数のモデルの組み合わせを考える) のときは結局クロスバリデーションが必要

## P.223 Author's Opinion
- 作業配分の参考
    - 特徴量作成: 5-8 割
    - ハイパーパラメータの調整: 変更時の影響をたまに見つつ, 本格的な調整は終盤
    - モデル選択
        - まずは GBDT
        - タスクの性質によってニューラルネットを検討
        - **(どういうこと?)** アンサンブル (複数のモデルの組み合わせ) を考える場合は他のモデルも作成
    - 理解が進んでくるたびにバリデーションの枠組みを再検討する
- モデルの種類・ハイパーパラメータ・特徴量のセットを入れ替えやすくするコードを書こう

## クロスバリデーション後の予測
- 2 つの手法
    - 各 fold で学習したモデルを保存し, それらのモデルの予測値の平均を取る
        - cf. P.225 の左の図
    - 学習データ全体に対して改めてモデルを学習させ, そのモデルで予測
- P.225 Author's Opinion: 手法のメリット・デメリット
    - 前者の手法
        - 追加学習が不要でバリデーションでスコアが見えていてわかりやすい
        - 各 fold で学習に使ったデータを合わせると学習データ全体なので,
          学習データ全体に対して学習させた場合と変わらない精度が出ると**見込める**
        - アンサンブルの効果が効く
            - ただしハイパーパラメータや特徴量を変えたモデルでの平均を取るとその効果は消える
        - 予測を fold 分くり返すからテストデータが大きいときは予測に時間がかかる
    - 後者の手法
        - 学習データ全体に対して学習させた方が少し精度がいい**という意見がある**
        - 学習データ全体に対して再学習させる時間がかかる
        - 学習データ数が違うのに同じハイパーパラメータでいいか懸念がある
            - ニューラルネットを使うとき, エポック数の設定が難しい
            - 各 fold と全体ではデータ数が違い, 同じエポックだけ学習させたときの進度が変わる

## 4.1.3 モデルに関連する用語とポイント

## 過学習 (オーバーフィッティング)
- 本の定義おかしくない?
    - 「学習データのランダムなノイズまで学習してしまい, 学習データではスコアがいいがそれ以外ではスコアが悪くなる現象」
    - というか, どう定義すると適切か?
    - 現象の定義としては単に「学習データではスコアがいいがそれ以外ではスコアが悪くなる現象」でいい?
    - これだと, 例えば[「持っている情報の量に比べて過剰に複雑なモデルを作ってしまうこと」](https://aizine.ai/overfitting0206/)が原因で,
      学習データの情報を吸い過ぎて学習データにだけ過剰適合してしまうことなども含められる.
    - 失敗原因 (の推測) を盛り込まない定義がよさそう.
- 逆 (と呼ばれる) 事例: アンダーフィッティング
    - これもおかしくない?
    - 「十分に学習データの性質が学習できていなくて学習データでもそれ以外でもスコアがよくない」
    - 「十分に学習データの性質が学習できていなくて」の前提がよけい.
    - 単に「学習データでもそれ以外でもスコアがよくない」でいいのではないか.
    - 「そもそもモデル自体が不適切」という事例も含めた定義をしたい.
    - 単に学習が足りないこともある.
    
- (コンペのスコアをもとに) ハイパーパラメータ調整の参考にする.
- スコアが違う場合の注意
    - バリデーションとテストでデータの分布が違うこともある
    - テストデータのレコード数が少ないこともある
    - もちろんバリデーションがうまくいっていない可能性もある

## 正則化 (regularization)
- 学習時にモデルの複雑さに対してペナルティを課すこと.
    - 過学習を防ぐ目的で導入する.
    - 本の過学習の定義の議論と整合性なくない?
        - 定義でモデルの複雑さに言及していない.
- ペナルティを上回るほど予測に寄与する複雑さだけが評価対象.
- 多くのモデルには正則化項がある
    - 正則化の強さを指定するハイパーパラメータの調整でモデルの複雑さを制御できる

## 学習データとバリデーションデータのスコアのモニタリング
- GBDT やニューラルネットは逐次的に学習が進んでいくモデル
    - 学習データとその目的変数の他,
      モニタリングする評価指標・バリデーションデータとその目的変数を与えることで,
      学習データとバリデーションデータのスコアの推移を見られる.
- モデルの学習でバリデーションデータも与えるのを基本にする

## アーリーストッピング
- バリデーションデータのスコアを見て一定期間スコアが上がらないとき途中で学習を打ち切る機能
    - ライブラリにも機能がある
- 学習を進めすぎて過学習になり汎化性能が落ちるのを防ぐ
- スコアのイメージについては P.228 図 4.6 を参照

## バギング
- 複数のモデルを組み合わせてモデルを作る方法：アンサンブルの具体例
- 同じ種類のモデルを並列に複数作り, その予測値の平均などを使って予測する
- それぞれのモデルでデータや特徴量をサンプリングして汎化性能を高める

## ブースティング
- 複数のモデルを組み合わせてモデルを作る方法: 同じ種類のモデルを直列的に組み合わせる
- それまでの学習による予測値を補正しつつ順に 1 つずつモデルを学習させる
- GBDT は名前の通りブースティングを使っている

## 4.2 分析コンペで使われるモデル
- 次のモデルを紹介する
    - GBDT (勾配ブースティング木)
    - ニューラルネット
    - 線型モデル
    - その他のモデル
        - k 近傍法 (k-nearest neighbor algorithm, KNN)
        - ランダムフォレスト (Random Forest, RF)
        - Extremely Randomized Trees (ERT)
        - Regularized Greedy Forest (RGF)
        - Field-aware Factorization Machines (FFM)

### 分析コンペでモデルを選ぶ観点
- 精度, 計算速度, 使いやすさ, 多様性でアンサンブルによる精度向上に寄与するか
- 最優先は精度
- 試行錯誤のために計算速度や使いやすさも重要
- モデル自体の精度は高くなくても, 
  精度が高い他のモデルと違う観点から予測しアンサンブルでの精度向上に寄与できることもある

### GBDT
- 精度・計算速度・使いやすさともに優れている
- ふつうは最初に試すモデル

### ニューラルネット
- 扱いが難しい
- タスクによって役に立つ

### 線型モデル
- 過学習しやすそうな特殊なコンペでは選択肢に上がる

### その他のモデル
- 多様性に貢献し, アンサンブルで精度を上げることを狙って使う

### 決定木ベースのモデル
- GBDT, ランダムフォレスト, Extremely Randomized Trees, Regularized Greedy Forest
- 1 つの決定木では十分な予測は難しいので複数を組み合わせる
- サポートベクターマシンは精度・計算速度の問題からあまり使わない
- cf. P.230 Author's Opinion の「モデルの選び方」

## 4.3 GBDT (勾配ブースティング木)

## 4.3.1 GBDT の概要
- 使いやすくて精度が高く, ふつうコンペの初手で作る
- 学習
    - 目的変数と予測値から計算される目的関数を改善するように、決定木を作ってモデルに追加
    - ハイパーパラメータで定めた決定木の本数の分だけくり返す
    - 2 本目以降の木は目的変数とそれまでに作った決定木による予測値の差に対して学習を進める
    - 木を作るうちにモデルの予測値が目的変数に合っていく: 作成させる決定木のウェイトは徐々に小さくなる
    - 予測値は予測対象のデータがそれぞれの決定木で属する葉のウェイトの和を取る
- 決定木の作り方
    - ランダムフォレスト: 決定木を並列に作る
    - GBDT: 決定木を直列に作る
        - それまでに作った決定木の予測値を新しい決定木の予測値を加えて少しずつ修正

## 4.3.2 GBDT の特徴
- 特徴量は数値: 特徴量の大小で分岐を振り分けるため
- 補完なしで欠損値が扱える: 欠損値があっても分岐のどちらかに振り分けられる
- 変数間の相互作用が反映される: 分岐のくり返しによる
- 著者の経験則
    - 精度が高い
    - パラメータチューニングしなくても精度が出やすい
    - 不要な特徴量を追加しても精度が落ちにくい
- 使いやすさの面からの特徴
    - 特徴量をスケーリングする必要がない: 決定木ではそれぞれの特徴量について値の大小関係だけが問題
    - カテゴリ変数を one-hot encoding しなくていい
        - 数値化するために label encoding は必要
    - 疎行列へのサポート

## 4.3.3 GBDT の主なライブラリ
- よく使われるライブラリ: xgboost, lightgbm, catboost
- 長く使われていて資料の多い xgboost で説明する
- lightgbm: 高速で, 2019/8 時点で xgboost よりも人気
- catboost: カテゴリ変数の扱いに工夫がある

## 4.3.4 GBDT の実装
- ch04/cho4-02-run_xgb.py

In [2]:
import numpy as np
import pandas as pd

# train_xは学習データ、train_yは目的変数、test_xはテストデータ
# pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）
train = pd.read_csv('kagglebook/input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('kagglebook/input/sample-data/test_preprocessed.csv')

# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

import xgboost as xgb
from sklearn.metrics import log_loss

# 特徴量と目的変数をxgboostのデータ構造に変換する
dtrain = xgb.DMatrix(tr_x, label=tr_y)
dvalid = xgb.DMatrix(va_x, label=va_y)
dtest = xgb.DMatrix(test_x)

# ハイパーパラメータの設定
params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
num_round = 50

# 学習の実行
# バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
# watchlistには学習データおよびバリデーションデータをセットする
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
model = xgb.train(params, dtrain, num_round, evals=watchlist)

# バリデーションデータでのスコアの確認
va_pred = model.predict(dvalid)
score = log_loss(va_y, va_pred)
print(f'logloss: {score:.4f}')

# 予測（二値の予測値ではなく、1である確率を出力するようにしている）
pred = model.predict(dtest)

[0]	train-error:0.12853	eval-error:0.15160
[1]	train-error:0.11533	eval-error:0.14600
[2]	train-error:0.10933	eval-error:0.13760
[3]	train-error:0.10533	eval-error:0.13640
[4]	train-error:0.09693	eval-error:0.13840
[5]	train-error:0.09467	eval-error:0.13640
[6]	train-error:0.08733	eval-error:0.12960
[7]	train-error:0.08493	eval-error:0.12440
[8]	train-error:0.07813	eval-error:0.12080
[9]	train-error:0.07373	eval-error:0.11720
[10]	train-error:0.06867	eval-error:0.11600
[11]	train-error:0.06493	eval-error:0.11640
[12]	train-error:0.06227	eval-error:0.11120
[13]	train-error:0.06053	eval-error:0.11160
[14]	train-error:0.05680	eval-error:0.11120
[15]	train-error:0.05040	eval-error:0.10480
[16]	train-error:0.04920	eval-error:0.10040
[17]	train-error:0.04640	eval-error:0.10160
[18]	train-error:0.04427	eval-error:0.10280
[19]	train-error:0.04347	eval-error:0.10120
[20]	train-error:0.03867	eval-error:0.10160
[21]	train-error:0.03653	eval-error:0.10040
[22]	train-error:0.03493	eval-error:0.0996

## 4.3.5 xgboost の使い方のポイント

## ブースター (booster)
- GBDT を使う場合はデフォルト値の gbtree でいい.
- gblinear: 線型モデル
    - 表現力が線型モデルと同じなのであまり使われない
- dart: 正則化に DART というアルゴリズムを使った GBDT
    - コンペによっては効果がある
    - 勾配ブースティングでは序盤に作成される木の影響が強い: 終盤に作成される木は瑣末な部分にフィットする
    - これを抑制するためにドロップアウトを GBDT に適用する手法
    - 決定木の作成ごとにドロップアウトのように一定の割合の木を存在しないとして学習させる

## 目的関数
- これを最小化するように学習を進める
- パラメータ objective を以下のように設定する
- 回帰: reg:squarederror を設定すると平均2乗誤差を最小化するように学習
- 二値分類: binary:logistic を設定すると logloss を最小化するように学習
- 多クラス分類: multi:softprob を設定して multi-class logloss を最小化するように学習

## ハイパーパラメータ
- 詳細は 6 章

## 学習データとバリデーションデータのスコアのモニタリング
- train メソッドの evals パラメータに学習データ・バリデーションデータを渡す
- 決定木を追加するごとに学習データ・バリデーションデータへのスコアを出力する
- デフォルトでは目的関数に応じて適した評価指標が設定される
- パラメータ `eval_metric` を指定するとモニタリングしたい評価指標への変更・複数の評価指標の設定ができる
- `train` メソッドの `early_stopping_round` パラメータの指定で, アーリーストッピングができる.
    - 注意: 予測時に ntree_limit パラメータを指定しないと最適ではなく学習が止まったところまでの木の本数で計算されてしまう

In [ ]:
# モニタリングをloglossで行い、アーリーストッピングの観察するroundを20とする
params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71,
          'eval_metric': 'logloss'}
num_round = 500
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
model = xgb.train(params, dtrain, num_round, evals=watchlist,
                  early_stopping_rounds=20)

# 最適な決定木の本数で予測を行う
pred = model.predict(dtest, ntree_limit=model.best_ntree_limit)

## Learning API と Scikit-Learn API のどちらを使うか
- どちらを使うは好み
- Scikit-Learn API は Learning API の train メソッドなどをラップしているので, 小回りが利かないこともある

## 4.3.6 lightgbm
- xgboost に比べて高速で, 精度は同程度
    - 決定木の分岐はヒストグラムベース
    - 深さ単位でなく葉単位での分岐の追加による精度の向上
        - estimator を見ると様子が分かる。
    - カテゴリ変数の分割の最適化による精度の向上

In [ ]:
import lightgbm as lgb
from sklearn.metrics import log_loss

# 特徴量と目的変数をlightgbmのデータ構造に変換する
lgb_train = lgb.Dataset(tr_x, tr_y)
lgb_eval = lgb.Dataset(va_x, va_y)

# ハイパーパラメータの設定
params = {'objective': 'binary', 'seed': 71, 'verbose': 0, 'metrics': 'binary_logloss'}
num_round = 100

# 学習の実行
# カテゴリ変数をパラメータで指定している
# バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
categorical_features = ['product', 'medical_info_b2', 'medical_info_b3']
model = lgb.train(params, lgb_train, num_boost_round=num_round,
                  categorical_feature=categorical_features,
                  valid_names=['train', 'valid'], valid_sets=[lgb_train, lgb_eval])

# バリデーションデータでのスコアの確認
va_pred = model.predict(va_x)
score = log_loss(va_y, va_pred)
print(f'logloss: {score:.4f}')

# 予測
pred = model.predict(test_x)

## 4.3.7 catboost
- カテゴリ変数の扱いに特徴的な工夫をしている
    - カテゴリ変数の target encoding: 自動的に数値に変換する
    - oblivious decision tree: 各深さの分岐の条件式が全て同じ決定木を使う, cf. P241, 図 4.14
    - ordered boosting: データ数が少ないときに使われるアルゴリズム
        - 遅いがデータ数が少ないときには精度がいい
- Author's Opinion
    - 速度が出ないので使われる機会は少ない
    - 相互作用が重要なときにいい精度が出る

## P.243 Column xgboost のアルゴリズムの解説
- 目的関数を変えればタスクは分類・回帰のどちらも同じ枠組みで計算できる
- 目的関数
    - 回帰なら二乗誤差
    - 二値分類なら logloss
    - マルチクラス分類なら multi-class logloss
- 以下の設定
    - レコード数: $N$, 添字: $i$
    - 決定木: $M$ 個, 添字: $m$

## a. 決定木によるブースティング
- モデルは決定木 (回帰木) によるブースティングで作る
    - 分類であっても回帰木を使う
    - 予測値を予測確率に変換して使う
- 決定木を逐次的に学習させる
    - $m$ 番目の木を学習させるときは $m-1$ 番目までの予測誤差を補正するように決める
    - 学習が進むと補正する必要性が小さくなり, ウェイトも小さくなる
- GBDT では各決定木でのレコードが属する葉のウェイトの合計がモデルの予測値
    - レコード $i$ の特徴量の組を $x_i$ とし, 
      ある決定木 $m$ でのレコードが属する葉のウェイトを $w_m(x_i)$ とすると,
      予測値は $\sum_{m=1}^M w_m(x_i)$ と表される.
    - 2 値分類では $\sum_{m=1}^M w_m(x_i)$ にシグモイド関数をあてた値が予測確率
- 流れは次の通り
    - 決定木を $M$ 本作り, 次のフローをくり返す
    - 分岐を何度も作ることで決定木を作る.
      分岐を作るためにはどの特徴量のどの値で分岐させるかを選ぶ.
    - どの特徴量のどの値で分岐させるかは次の通り.
        - 全ての候補を調べ, 分岐させる
        - 目的関数が 1 番減る葉のウェイトを選ぶ
    - 作った決定木にもとづいて予測値を更新する
        - 例: あるレコードが分岐でウェイト $w$ の葉に落ちる場合,
          学習率を $\eta$ として,
          レコードの予測値には $w \eta$ を加える.

## b. 正則化された目的関数
- 目的変数 $y_i$, 予測値 $\dot{y}_i$ としたときの目的関数を $l(y_i, \dot{y}_i)$ とする.
- それぞれの決定木を $f_m$ として決定木に対して罰則が計算される正則化項を $\Omega(f_m)$ とする.
- $T$ を木の葉の数, $j$ を葉の添字, $w_j$ を葉のウェイトとする.
    - 正則化項は $$\Omega(f_m) = \gamma T + \frac{1}{2} \lambda \sum_{j} w_j^2 + \alpha \sum_{j} |w_j|$$
- 目的関数は次の通り: $$L = \sum_{i=1}^N l(\hat{y}_j, y_i) + \sum_{m=1}^M \Omega(f_m)$$.
- 正則化はモデルをできるだけシンプルにするために導入する
    - xgboost では正則化項まで含めて計算して葉のウェイトを決める
    - 以下では正則化項は省略

## c. 決定木の作成
- それぞれ 1 つの葉から始めて分岐をくり返して作る
- 対象とする点での勾配を使って決定木の分岐の仕方を決める

## c-1. 葉の最適なウェイトと分岐で得られる目的関数の計算
- どの特徴量で分岐させるか, どの値の大小で分岐させるかを決めると分岐先の葉に含まれるデータの集合が得られる
- それらの与えるべき最適なウェイトとそのときの目的関数の変化を求めたい
- それまでの決定木による予測値が $\hat{y}_i$,
  分岐を決めたときのある葉のレコードの集合を $I_j$,
  ウェイトを $w_j$ とする.
- レコードの集合の目的関数の和は $$L_j = \sum_{i \in I_j} l(y_i, \hat{y}_i + w_j)$$.
- これを 2 次近似する: それぞれのレコードで目的関数を 2 次関数とし, その和を最適化する.
- 予測値 $\hat{y}_j$ まわりの勾配を $g_i = l_{\hat{y}_i}$,
  2 階の微分係数を $h_i = l_{\hat{y}_i \hat[y]_i}$ とすると,
  目的関数の和は次の通り
    - $$\tilde{L}_j = \sum_{i \in I_j} (l(y_i, \hat{y}_i) + g_i w_j + \frac{1}{2} h_i w_j^2)$$.
    - 定数部分はウェイトの決め方に関与しないので除く
    - $$\tilde{L}'_j = \sum_{i \in I_j} (g_i w_j + \frac{1}{2} h_i w_j^2)$$.
- 次のようにウェイトを決めれば目的関数 $\tilde{L}'_j$ は最小になる

\begin{align}
 w_j = - \frac{\sum_{i \in I_j} g_i}{\sum_{i \in I_j} h_i}, \quad
 \tilde{L}'_j = \frac{1}{2} \frac{(\sum_{i \in I_j} g_i)}{\sum_{i \in I_j} h_i}.
\end{align}

- $g_i$, $h_i$ が決まれば目的関数の値が決まり, 分岐したときの目的関数の減り方も決まる
- 分岐前の目的関数を $\tilde{L}'_j$,
  分岐後の左右の葉の目的関数を $\tilde{L}'_{jL}, \tilde{L}'_{jR}$ とすると,
  分岐による目的関数の減少は $\tilde{L}'_j - (\tilde{L}'_{jL} - \tilde{L}'_{jR})$.

### 2 乗誤差の場合の目的関数の計算
- 二乗誤差なら計算しやすくなる
- 目的関数は $l(y_i, \hat{y}_i) = \frac{1}{2} (\hat{y}_i - y_i)^2$.
    - 勾配: $g_i = l_{\hat{y}_i} = \hat{y}_i - y_i$
    - 二階の微分係数: $h_i = l_{\hat{y}_i \hat{y}_i} = 1$.
- これを 2 次近似目的関数の和に代入

\begin{align}
 \tilde{L}_j
 &= \sum_{i \in I_j} \left( \frac{1}{2} (\hat{y}_i - y_i)^2 + (\hat{y}_i - y_i) w_j + \frac{1}{2} w_j^2 \right) \\
 &= \sum_{i \in I_j} \left( \frac{1}{2} ((\hat{y}_i + w_j) - y_j)^2 \right)
\end{align}

- 括弧内は $l(y_i, \hat{y}_i + w_j)$ であることに注意.
- 目的関数 $\tilde{L}'_j$ を最小にするウェイトは次の通り.

\begin{align}
 w_j = - \frac{\sum_{i \in I_j} g_i}{\sum_{i \in I_j} h_i}
 = - \frac{\sum_{i \in I_j} (\hat{y}_i - y_i)}{\sum_{i \in I_j} 1}
\end{align}

## c-2 分岐の特徴量と基準値の決定
- どの特徴量で分岐させるか, どの値の大小で分岐させるか
    - 全ての候補を調べて正則化された目的関数が 1 番小さくなるように選ぶ
    - データが多い場合は一定の分位点だけを候補として調べる
    - 疎なデータに対しては効率的に計算できる

## d. 過学習を防ぐ工夫 (正則化以外の方法)
- 学習率
    - 上記で求めた最適なウェイトまで一気に予測値を補正してしまうと過学習になる
    - (**どういうこと?**) 実際には各決定木で求めたウェイトの一定割合を適用し, 少しずつ補正
    - 率はパラメータ `eta` で指定する.
- サンプリング
    - 決定木を作るときに特徴量の列をサンプリングする
    - 割合は `colsample_bytree` で指定する
    - それぞれの決定木を作るときに学習データの行もサンプリングする
    - 割合はパラメータ `subsample` で指定
- データが少なすぎる葉を作らない
    - 葉の構成に最低限必要な 2 階微分係数の値を `min_child_weight` で指定
- 決定木の深さの制限
    - 木の深さの最大値は `max_depth` で制限できる
    - 深くすると複雑なモデルになる表現力は上がるが過学習しやすくなる

## 4.4 ニューラルネット

## 4.4.1 ニューラルネットの概要
- テーブルデータに対してよく使われる構造を考える。
    - いわゆる深層学習で使われる 10 層以上あるようなニューラルネットではなく、中間層が 2-4 層程度の全結合層からなる多層パーセプトロン（Multi Layer Perceptron、MLP）
    - 中間層が 2 層の多層パーセプトロンは P.247 図 4.15
- 多層パーセプトロンの概要
    - 入力層：特徴量がインプット
    - 中間層：前の層の値をウェイトで重みづけした和を取って結合し、そのあと活性化関数を適用する。
      活性化関数には ReLU（Rectified Linear Unit）がよく使われる。
    - 出力層：前の層の値をウェイトで重みづけした和を取り結合する。
      そのあとタスクに合わせて活性化関数を適用する。
      - 出力層の活性化関数の選び方
      - 回帰：恒等関数
      - 二値分類：シグモイド関数
      - マルチクラス分類：ソフトマックス関数

## ユニット
- 層の各要素をユニットという。
- 入力層のユニット数は特徴量と同じ数。
- 中間層のユニット数はハイパーパラメータで設定。
- 出力層のユニット数は回帰・二値分類なら 1 つ、多クラス分類ならクラス数。

## 中間層の計算
- ある層のユニット $i$ の出力値 $z_i$ の計算は次の通り。
- 前の層からの各ユニットの出力の和を取って結合する：$u_i = \sum_{j} z'_j w_{i,j}$
    - 出力は前の層のユニット $j$ の出力とウェイトの積で定義する：$z'_j w_{i,j}$
    - バイアスの記述は省略：「モデルと学習データの平均的なずれ」をバイアスと呼ぶ。
- 活性化関数 ReLU を適用し、出力は $z_i = \max (u_i,0)$。
    - $f(x) = \max(x,0)$ を ReLU という。
- 活性化関数を適用して層を重ねると非線型性が表現できる。

## P.248 いろいろな活性化関数
- 出力層にシグモイド関数・ソフトマックス関数を適用すると、出力値は二値分類・多クラス分類で確率を表す値になる。
- シグモイド関数は出力値を $(0,1)$ に制限する：$$f(x) = \frac{1}{1 + e^{-x}}.$$
- ソフトマックス関数も出力値を $(0,1)$ に制限し、各出力の和が 1 になるように変換する：クラス数 $K$ に対して次のように定義する。$$f_i(x_1,\dots,x_k) = \frac{e^{x_i}}{\sum_{k=1}^K e^{x_k}}.$$
    - 自分用メモ：指数の肩に負号はつけない模様。

## P.248 学習
- 学習するのは中間層・出力層のウェイト。
- 学習には**勾配降下法**を使う：誤差を出力層から入力層に伝播させ、ウェイトを更新する**誤差逆伝播法**（バックプロパゲーション）を使う。
- 勾配降下法の最適化アルゴリズム（オプティマイザー）がいくつかあり、ハイパーパラメータとして指定する。
- 学習データをミニバッチと呼ばれる少数のサンプルに分け、ミニバッチごとに層のウェイトを更新する。
- 1 エポックの定義：学習データの最後まで学習する。
- 十分に学習が進むまでエポックくり返す。
- 確率的勾配降下法：ミニバッチごとに更新する手法。
    - 学習データ全体を計算するごとにウェイトを更新するよりも、計算効率が高くなりやすく、局所解にも陥りにくい。

## P.249 4.4.2 ニューラルネットの特徴
- 特徴量は数値
- 欠損値は扱えない
- 非線型性や変数間の相互作用が反映できる
- 特徴量を標準化などでスケーリングしなければならない
    - 特徴量の大きさが揃っていないと学習がうまくいかないことがある
- ハイパーパラメータの調整が難しい
    - きちんと調整しないと精度が出ない
    - 過学習も学習が進まないこともある
- 多クラス分類に (比較的) 強い
    - 構造上, 多クラス分類を自然にモデリングできる
    - GBDT と遜色ない精度が出ることもある
- GPU での高速化
    - GPU はニューラルネットで必要な行列演算に適している
- GBDT よりもモデリングやチューニングの手間がかかる

## 4.4.3 ニューラルネットの主なライブラリ
- keras, pytorch, chainer, tensorflow
    - chainer は死んでしまった
- この本は keras で解説
- keras は tensorflow などのライブラリをバックエンドとし, 扱いやすい API を提供するラッパー

## P.250 4.4.4 ニューラルネットの実装
- サンプルデータに対して keras でモデリング
- ニューラルネットで学習させるデータはカテゴリ変数を one-hot encoding している

In [ ]:
# ch04/ch04-04-run_nn.py から
from keras.layers import Dense, Dropout
from keras.models import Sequential
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler

# データのスケーリング
scaler = StandardScaler()
tr_x = scaler.fit_transform(tr_x)
va_x = scaler.transform(va_x)
test_x = scaler.transform(test_x)

# ニューラルネットモデルの構築
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(train_x.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=['accuracy'])

# 学習の実行
# バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
batch_size = 128
epochs = 10
history = model.fit(tr_x, tr_y,
                    batch_size=batch_size, epochs=epochs,
                    verbose=1, validation_data=(va_x, va_y))

# バリデーションデータでのスコアの確認
va_pred = model.predict(va_x)
score = log_loss(va_y, va_pred, eps=1e-7)
print(f'logloss: {score:.4f}')

# 予測
pred = model.predict(test_x)

## P.251 keras の使い方のポイント

## P.251 目的関数
- モデルのコンパイル時にパラメータ `loss` に目的関数を設定すると,
  それを最小化するように学習が進む.
- 回帰の場合: `mean_squared_error` を設定すると平均 2 乗誤差を最小化する
- 二値分類: `binary_crossentropy` を設定すると `logloss` を最小化するように学習する.
- 多クラス分類: `categorical_crossentropy` を設定すると `multi-classlogloss` を最小化するように学習する.
    - この本、多クラス分類とマルチクラス分類が両方出てくるっぽく、編集者が仕事をしていない疑惑

## P.252 ハイパーパラメータ
- ニューラルネットはハイパーパラメータの種類が決まっていない。
    - cf. 多くのモデルではハイパーパラメータの種類が決まっている
- ニューラルネットは層の構成など自由にできる部分が多い
    - ニューラルネットの難しさの一因
    - 調整しやすいようにコードを書こう
    - まずはパラメータに応じて中間層の数やユニット数が違うネットワークを作るコードを書く
    - そのあとパラメータを設定して調整する
    - 具体的なハイパーパラメータ
        - 詳細は 6 章
        - オプティマイザの種類
        - 学習率
        - 中間層の数
        - ユニット数
        - ドロップアウトの強さ: 次に説明

## ドロップアウト
- 階層の深いニューラルネットを精度よく最適化するために Hinton らによって提案された手法
- 複数のネットワークの学習結果の平均と同じ効果があるとされていて過学習を防ぐ
- 無視する割合をハイパーパラメータとして指定する
- [参考](http://sonickun.hatenablog.com/entry/2016/07/18/191656)
    - 学習時にネットワークの自由度を強制的に小さくして汎化性能を上げ過学習を避ける
    - ニューラルネットワークを学習するとき,
      ある更新で層の中のノードのうちのいくつかをランダムに無効にする:
      そもそも存在しないかのように扱って学習する.
    - 次の更新では別のノードを無効にして学習を行うことを繰り返す.
    - 隠れ層では一般的に 50% 程度を無効するといいとされる.

## 学習データとバリデーションデータのスコアのモニタリング
- エポックごとに学習データとバリデーションデータのスコアを出力する
- アーリーストッピングは後述のコールバックで

## コールバック
- 学習時, ミニバッチの処理ごとやエポックごとに指定した処理を走らせる
- 次のようなときに使う
    - アーリーストッピング
    - モデルの定期的な保存: バリデーションデータでの評価がいいモデルを残す
    - 学習率のスケジューリング・調整
    - ログ・可視化
- 次の例はコールバックでアーリーストッピングしている

In [ ]:
# ch04/ch04-04-run_nn.py
from keras.callbacks import EarlyStopping

# アーリーストッピングの観察するroundを20とする
# restore_best_weightsを設定することで、最適なエポックでのモデルを使用する
epochs = 50
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(tr_x, tr_y,
                    batch_size=batch_size, epochs=epochs,
                    verbose=1, validation_data=(va_x, va_y), callbacks=[early_stopping])
pred = model.predict(test_x)

## embedding layer
- 正の整数を密な数値ベクトルに変換する層
    - (**疑問**) 「密な」というのはどういうこと? 整数を 0 が少ない数列・ベクトルに置き換える。
    - 参考：https://www.tensorflow.org/tutorials/text/word_embeddings?hl=ja
- モデルの最初の層としてしか指定できない
    - カテゴリ変数を入力にするときに使える
    - 二値ではないカテゴリ変数はもともと one-hot encoding で前処理していた.
    - label encoding から embedding layer を使う手法が出てきた
- 自然言語対応
    - `Word2Vec` や `Glove` などの学習済み embedding (単語を対応する密な数値ベクトルで表現したもの) をウェイトに設定できる

## batch normalization 層 (BN 層)
- ミニバッチごとに標準化して各層の出力のばらつきを抑える
    - 効果が高く広く使われている
    - 学習に効果的な理由は本の P253、注 29 の文献参照
- BN 層への入力
    - ミニバッチごとに標準化: 平均を 0, 標準偏差を 1 にする
    - 標準化された値 $\hat{x}$ に対して $\gamma \hat{x} + \beta$ に変換されて出力される
        - この $\gamma$ と $\beta$ は BN 層で学習されるパラメータ
    - それぞれの入力ごとに計算され, $\gamma, \beta$ も入力の数だけある
- - 学習中の BN 層への入力の平均・標準偏差は保持される
    - 予測ではこれらで標準化する: ミニバッチの選び方で結果が変わらないようにしている

## 4.4.6 参考になるソリューション: 多層パーセプトロン
- ニューラルネットの難しさ: 選択肢が多くモデル作りの自由度が高い
    - 層をどう作るか, オプティマイザーに何を使うか
    - Kaggle の過去のソリューションを基礎にするといい
- 以下のソリューションには多層パーセプトロンのモデルがある
    - [Recruit Retaurant Visitor Forecasting](https://github.com/dkivaranovic/kaggledays-recruit)
    - [Corporation Favorita Grocery Sales Forecasting](https://www.kaggle.com/shixw125/1st-place-nn-model-public-0-507-private-0-513)
    - [Otto Group Product Classification Challenge](https://github.com/puyokw/kaggle_Otto)
    - [Home Depot Product Search Relevance](https://github.com/ChenglongChen/Kaggle_HomeDepot)
        - hyperopt で自動的にパラメータをチューニングしてモデルを作っている
    - [https://www.kaggle.com/c/mercari-price-suggestion-challenge/discussion/50256]
        - 商品タイトルや説明文など自然言語中心のデータからその商品価格を予測する
        - 疎なデータをインプットにする多層パーセプトロンのソリューションがメイン

## P.255 4.4.7 参考になるソリューション: 最近のニューラルネットの発展
- テーブルデータ対象の場合, 以前はほぼパーセプトロンだった.
- 最近は RNN (リカレントニューラルネット) が使われるようになってきた
- 人手での特徴量作成による GBDT と, あまり特徴量を生成しないニューラルネットが遜色ない精度になっているケースもある
- 例
    - [Instacart Market Basket Analysis](https://github.com/sjvasquez/instacart-basket-prediction)
        - 顧客の時系列的な注文データから次のどの商品が購入されるか予測する.
        - RNN・CNN (たたみ込みニューラルネット) でモデルを作り, アンサンブルしている
    - [Web Traffic Time Series Forecasting](https://github.com/sjvasquez/web-traffic-forecasting)
        - Wikipedia の記事の日ごとの閲覧数を予測する
        - WaveNet という音声波形生成のためのニューラルネットをベースにしている
    - [Mercari Price Suggestion Challenge](https://github.com/ChenglongChen/tensorflow-XNN)
        - DeepFM という Factorization Machine というモデルの組み込みをネットワーク構造に組み込んだニューラルネットがベース
        - [Guo, Tang, Ye, Li, He, 2017, DeepFM: A Factorization-Machine based Neural Network for CTR Prediction](https://arxiv.org/abs/1703.04247)

## P.256, 4.5 線型モデル

## P.256, 4.5.1 線型モデルの概要
- 線型モデル: シンプルなモデル
- 単体での精度は高くない
- GBDT やニューラルネットに勝つことはまずない
- アンサンブルのうちの 1 つやスタッキングの最終層に使う
    - スタッキング: 効率的かつ効果的に 2 つ以上のモデルを組み合わせて予測する方法
    - cf. P360, 7.3
- Author's Opinion
    - 過学習しやすいデータで活躍する: データ不足だったりノイズが多かったり.
    - Kaggle の Two Sigma Financial Modeling Challenge や Walmart Recruting II: Sales in Stormy Weather で使われている
- 回帰タスク
    - 次のような線型回帰モデルを使う.
    - 後述の $L^1$ 正則化を使う線型回帰モデルを Lasso, $L^2$ 正則化を使う線型回帰モデルを Ridge という.
    - 予測値 $y$, 特徴量 $x_i$ に対して $y =  b_0 + \sum_{k=1}^n b_k x_k$ を考え,
      各係数 $b_k$ を学習させる.
- 分類タスク
    - ロジスティック回帰モデルが使われる
    - ロジスティック回帰
        - 線型回帰にシグモイド関数を適用
        - 予測値の取り得る値を $(0,1)$ に制限, 確率予測ができるようにした
        - $y' = b_0 + \sum_{k=1}^n b_k x_k$, $y = 1 / (1 + e^{-y'})$.
- 線型モデルでの正則化: 係数の絶対値が大きいほど罰則を与える
    - 大きすぎる係数での学習データへの過剰適合を防ぐ
    - 係数の絶対値に比例する罰則を $L^1$ 正則化, 2 乗に比例する罰則を $L^2$ 正則化

## P.257, 4.5.2 線型モデルの特徴
- 特徴量は数値
- 欠損値は扱えない
- GBDT やニューラルネットと比較して精度は高くない
- 非線型性を表現するためには明示的に特徴量を作る必要がある
    - 特徴量 $x_f$ が予測値に対して $\log x_f$ 程度に影響していることを表すには $\log x_f$ という特徴量を作る必要がある
- 相互作用を表現するためには明示的に特徴量を作る必要がある
    - フラグ 1 とフラグ 2 の関連性を見たければ, 「フラグ 1 かつフラグ 2」といった特徴量が必要.
- 特徴量は標準化すべし
    - 正則化の効き方が特徴量によって変わってしまい, 学習がうまくいかないことがある
- 特徴量を作るときに丁寧な処理が必要
    - 非線型性や相互作用を表す特徴量をわざわざ作る必要あり
    - 最大・最小を制限
    - binning する
    - 上記を組み合わせる
    - いろいろな変換・処理が必要
- $L^1$ 正則化をするとき, 予測に寄与しない特徴量の係数が 0 になる
    - 逆にこの性質を使って線型モデルを特徴選択に使うこともある

## P.257, 4.5.3 線型モデルの主なライブラリ
- scikit-learn の linear_model モジュール
- vowpal wabbit
- 学習は速い
- 以下では scikit-learn の linear_model で説明する
- 次のクラスを選ぶといい
    - 回帰タスクは Ridge: Ridge では $L^2$-正則化を適用.
      $L^1$-正則化を適用する Lasso や $L^1$-正則化・$L^2$-正則化がともに使える ElasticNet を使ってもいい.
    - 分類タスクは LogisticRegression: デフォルトでは $L^2$-正則化.

## P.258, 4.5.4 線型モデルの実装
- サンプルデータに対して scikit-learn の LogisticRegression モデルでモデリングする
- カテゴリ変数は one-hot encoding しておく

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler

# データのスケーリング
scaler = StandardScaler()
tr_x = scaler.fit_transform(tr_x)
va_x = scaler.transform(va_x)
test_x = scaler.transform(test_x)

# 線形モデルの構築・学習
model = LogisticRegression(C=1.0)
model.fit(tr_x, tr_y)

# バリデーションデータでのスコアの確認
# predict_probaを使うことで確率を出力できます。(predictでは二値のクラスの予測値が出力されます。)
va_pred = model.predict_proba(va_x)
score = log_loss(va_y, va_pred)
print(f'logloss: {score:.4f}')

# 予測
pred = model.predict(test_x)

## P.259, 4.5.5 線型モデルの使い方のポイント

## P.259 目的関数
- 基本的にはモデルによって最小化する目的関数が決まっている
- 回帰 (Ridge モデルなど): 平均 2 乗誤差を最小化するように学習
- 分類 (LogisticRegression モデル)
    - 二値分類: logloss を最小化
    - マルチクラス分類: one-vs-rest: あるクラスとそれ以外のクラスの二値分類をくり返す
        - multi-class logloss を最小化する方法で学習するオプションもある

## P.259 ハイパーパラメータ
- チューニングが必要なパラメータは基本的に正則化の強さを表す係数しかない

## P.260, 4.6 その他のモデル
- アンサンブルのモデルの 1 つとして使われることが多い

## P.260, 4.6.1 k 近傍法 (k-nearest neighbor algorithm, kNN)
- レコード間の距離をそれらの特徴量の差で定義
- 距離が最も近い $k$ 個のレコードの目的変数から回帰・分類する
- 利用モジュール
    - scikit-learn の neighbors モジュールの KNeighborsClassifier, KNeighborsRegressor クラス
- デフォルト
    - 距離はユークリッド距離
    - 回帰では最も近い $k$ 個のレコードの平均が予測値
    - 分類では最も近い $k$ 個のレコードで最も多いクラスが予測値
- 値のスケールが大きい特徴量が重視されすぎないようにする
    - 特徴量に標準化などのスケーリングが必要

## P.260, 4.6.2 ランダムフォレスト (Random Forest, RF)
- 決定木の集合で予測する
- GBDT と違い, 並列に決定木を作る: (P.261, 図 4.16)
- それぞれの決定木の学習でレコードや特徴量をサンプリングして与えて多様な決定木を作る
    - さらにアンサンブルして汎化性能が高い予測をする
- モデルの作り方
    - 学習データからレコードをサンプリングして抽出
    - 抽出レコードから学習して決定木を作る
        1. 分岐を作るとき特徴量の一部をサンプリングして抽出して特徴量の候補とする
        2. 特徴量の候補からデータを最もよく分割する特徴量と閾値を選んで分岐にする
        3. (1) から (2) を作る決定木の本数だけ並列に処理する
- scikit-learn の ensemble モジュールの RandomForestClassifier, RandomForestRegressor クラスを使う
    - 以下ではこれらのデフォルトパラメータで説明する

## P.261 決定木作成のポイント
- 分岐
    - 回帰タスク: 二乗誤差が最も減少するようにする
    - 分類タスク: ジニ不純度が最も減少するようにする
- 決定木ごとに元の個数と同じだけのレコードを復元抽出するブートストラップサンプリングをする
    - ブートストラップサンプリングでは重複して抽出されるレコードがある
    - 一方, 平均して 1/3 程度が抽出されないレコードになる
- 分岐ごとに特徴量の一部をサンプリングした候補から分岐の特徴量を選ぶ
    - 回帰タスク: サンプリングせずに全てを候補にする
    - 分類タスク: 特徴量の個数の平方根の個数だけ抽出して候補にする

## P.261 ランダムフォレストのポイント
- 決定木の本数と精度の関係
    - 決定木を並列に作るため, GBDT と違い, 決定木の本数が増え過ぎて精度が悪くなることはない
    - ある程度増やすと精度が上がらなくなる
    - 決定木の本数は計算時間と精度のトレードオフ
- out-of-bag
    - ブートストラップサンプリングで抽出されないレコードのこと
    - out-of-bag のレコードを使うことで, バリデーションデータを用意しなくても汎化性能が見積もれる
- 予測確率の妥当性
    - 分類タスク
        - GBDT ではウェイトに基づいた予測確率の logloss を最小化
            - ランダムフォレストではジニ不純度を最小化しようとする各決定木の予測を平均する
    - ランダムフォレストの方法では予測確率の妥当性は保証されず, 歪む
        - cf. P.95, 2.5.4 予測確率とその調整

## P.262 4.6.3 Extremely Randomized Trees (ERT)
- ランダムフォレストとほぼ同じ方法でモデルを作る
- 分岐を作るときに違いがある
    - ランダムフォレスト: それぞれの特徴量で最もよくデータを分割できる閾値を使う
    - ERT: ランダムに設定した閾値を使う
- ランダムフォレストより過学習しづらい**らしい**
- scikit-learn の ensemble モジュールの ExtraTreesClassifier, ExtraTreesRegressor クラスを使う

## P.262 4.6.3 Regularized Greedy Forest (RGF)
- 目的関数に正則化項を明示的に含める点で GBDT と同じ
- 決定木を作成・成長させる方法が違う
- 目的関数が小さくなるように次の操作をくり返して決定木の集合を作る
    - 葉を分岐させる
    - 新しい木を作る
    - いままでに作った決定木全体に対して葉のウェイトを修正する
        - 計算コストが高い操作なので, 一定数の葉の分岐や木の作成ごとに定期的に実行
- Regularized Greedy Forest ライブラリを使う: pip install rgf_python

## P.263 4.6.5 Field-aware Factorization Machines
- Factorization Machines (FM) を発展させたモデル
- レコメンドのタスクと相性がいい
- ライブラリ libffm を使う: xlearn もある
- 説明, P263, Information
    - 問題: ユーザー・商品・ジャンルがカテゴリ変数で, 組み合わせへの評価の値が目的変数
    - ユーザー・商品・ジャンルを one-hot encoding して表す
        - 「ユーザー数・商品数・ジャンル数」が特徴量数で, 疎なデータとして表せる
        - cf. P.263, 図 4.17
    - FM は特徴量同士の相互作用を特徴量に対応するベクトルの内積で表現する線型モデルの亜種
        - それぞれの特徴量はその性質を表す要素数 $k$ のベクトルを持つ
        - これらのベクトルが学習対象
        - 要素数 $k$ はハイパーパラメータ
        - 予測値: $i$ 番目の特徴量のベクトル $v_i$ に対して次のように定義する.
          $$y = w_0 + \sum_{i=1}^{n_f} w_i x_i + \sum_{i=1}^{n_f} \sum_{j=i+1}^{n_f} \langle v_i, v_j \rangle x_i x_j.$$
        - $w_0$ は定数項, $w_i$ はウェイト, $n_f$ は特徴量の数
        - 評価した時点からの経過期間などカテゴリ変数だけではなく数値変数を特徴量に持てる
    - FFM はこれの拡張
        - 組み合わせの相手の種類ごとに異なるベクトルを持たせる
        - その種類をフィールドと呼ぶ: いまの例ではユーザー・商品・ジャンル
        - 次の式では組み合わせ相手の $j$ 番目の特徴量が属するフィールドが $f_j$ の場合に使われる,
          $i$ 番目の特徴量のベクトルを $v_{i, f_j}$ と表す.
          $$y = w_0 + \sum_{i=1}^{n_f} w_i x_i + \sum_{i=1}^{n_f} \sum_{j=i+1}^{n_f} \langle v_{i, f_j}, v_{j, f_i} \rangle x_i x_j.$$
        - FM: 学習対象のベクトルの個数は特徴量数
        - FFM: 特徴量数×フィールド数
        - 特定のフィールドとの関係性だけ考えればいいので, 必要なベクトルの要素数は少なくなる

## P.265 4.7 モデルのその他のポイントとテクニック
- モデルと扱うときに困る点の対応やその他のテクニックを紹介する

## P.265 4.7.1 欠損値がある場合
- GBDT は問題なく扱える
- 逆に欠損値を扱えないモデルでは欠損値を埋める必要がある
- 欠損値を扱う方法は「3.3 欠損値の扱い」参照

## P.265 4.7.2 特徴量の数が多い場合
- 多過ぎると学習がいつまで経っても終わらない・メモリ不足で学習できないなどの問題が起きる
- 余計な特徴量のせいでモデルの精度が上がらないこともある
- GBDT の場合
    - 学習さえできればそれなりの結果が出ること**も**ある
    - 少しずつ特徴量を増やしながらどのくらいまで学習できるか試してみるといい
    - 特徴量が数千あっても, データが疎の場合や二値しか持たない場合は分岐の候補が多くない: 学習できて精度が出るかもしれない
- よけいな特徴量がないに越したことはない
- 精度の寄与が少ない特徴量を落としたい場合は「6.2 特徴選択および特徴量の重要度」を参照

## P. 265 4.7.3 目的変数に 1対1 で対応するテーブルでない場合
- 教師あり学習のためには以下の形になっている必要がある
    - $n_{tr}$ はレコード数, $n_f$ は特徴量の数.
    - 学習データは $n_{tr} \times n_f$ の行列
    - 目的変数は $n_f$ 個の配列
- 最初からこの形ならいい
    - コンペによっては目的変数 1 つに複数行のレコードが対応することがある
- Kaggle の Walmart Recruiting: Trip Type Classification
    - 目的変数に対応するものとして複数行からなる購入した商品の履歴が与えられた
    - このままでは予測できない
    - 行数・購入量の合計・ある商品の有無など目的変数に 1:1 に対応させた特徴量に変換する必要あり
- 特徴量の作り方: 3.8・3.9 節参照

## P.266 4.7.4 pseudo labeling
- 目的変数のないデータについても学習に使える半教師あり学習の手法の 1 つ
- テストデータに対する予測値を目的変数の値とみなし, 学習データに加えて再度学習するテクニック
- 画像系のコンペで活用する例がいくつかある
    - テーブルデータで効果がある例もある
    - テストデータ数が学習データ数より多い場合にも有効
        - 目的変数がなくてもテストデータの情報を使いたい場合などもある
- 次のように実行する
    1. 学習データで学習し, モデルを作る
    2. 上の 1. で作ったモデルによってテストデータを予測する
    3. 学習データに 2. の予測値を目的変数としてテストデータを加える
         - 使う予測値を pseudo label と呼ぶ
    4. テストデータが追加された学習データで再度学習し, モデルを作る
    5. 上の 4. で作ったモデルでテストデータを予測し, 最終的な予測値にする
- 細かな手法がいろいろあり, 工夫もいろいろある
    - 学習データの質を保つため, 例えば分類タスクでは予測確率が十分高いテストデータだけを加える
    - 複数のモデルのアンサンブルによる予測値を pseudo label として使う
    - テストデータをいくつかのグループにわけ, あるグループの最終的な予測値はそのグループ以外のテストデータを加えて学習したモデルで作る

## P.267 COLUMN 分析コンペ用のクラスやフォルダの構成
- GitHub のサンプルも参照
- コンペでは綺麗に書きすぎるのも問題: スピードを重視する
    - ファイル名を a01_run_xgb.py, a02_run_nn.py のように連番で書く
    - ファイルはあまり分割せず, 修正する場合はファイルをコピーして修正することで, 過去のコードを残して再実行しやすくする
- 著者のスタイル: ある程度クラスを整理して進める
    - Model クラス
        - xgboost や scikit-learn の各モデルをラップしたクラスで, 学習・予測する
        - scikit-learn の BaseEstimator を継承すると行儀がいいかもしれない
        - 状況に合わせていろいろ工夫する
    - Runner クラス
        - クロスバリデーションを含めて学習・予測の一連の流れを実行する
        - データも読み込む
        - Model クラスを保持して学習・予測は Model クラスに実行させる
    - Util クラス, Logger クラス
        - ユーティリティメソッド: ファイルの入出力など
        - ログの出力・表示: ファイルとコンソールへの出力
            - 異常終了したときの原因把握, 処理にかかる時間の見積もりなど
        - 計算結果の出力・表示: 各モデルのバリデーションのスコアをファイルとコンソールに出力し, 集計する
    - フォルダ構成
        - 本の P.268 の注 48 に紹介された記事も参考にする
        - コードは code, code-analysis ディレクトリにだけ保存する
        - input: train.csv, test.csv などの入力ファイルを入れる
        - code: 計算用のコード
        - code-analysis: 分析用のコードや Jupyter Notebook
        - model: モデルや特徴量を保存する
        - submission: 提出用ファイルを保存する
    - a. Model クラス
        - run のクロスバリデーションの各 fold ごとにインスタンスを作る
        - クラス生成時には run の名前とどの fold かを組み合わせた名前 (xgb-param1-fold1) を渡す
        - 保存先のパスに使ってモデルを保存し, 読み込む